In [36]:
import sys
print("Python:", sys.version)

Python: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from skimage.measure import label, regionprops
from skimage.morphology import disk, binary_opening, binary_closing, octagon
from skimage.morphology import remove_small_objects
from tqdm import tqdm
from scipy.spatial.distance import pdist
import math

In [2]:
def usFOV(img,perc_header=0.075):
    img = np.array(img)

    # Calculate the new position of y.
    new_y0 = int(np.round(img.shape[0] * perc_header))

    # Convert to grayscale and binarize
    gray = cv2.cvtColor(img[new_y0-1:, :, :], cv2.COLOR_RGB2GRAY)
    binary = gray > 0

    # Label the regions and keep only the largest one.
    labeled_img = label(binary)
    largest_region = max(regionprops(labeled_img), key=lambda region: region.area)
    binary = labeled_img == largest_region.label

    # Calculate the size of the structuring element.
    tsl = disk(round(min(binary.shape) * 0.09))

    # Apply the morphological opening operation.
    box1 = binary_opening(binary, tsl)

    # Label the regions again.
    box1 = label(box1)

    # Get the coordinates of the bounding box.
    box1_props = max(regionprops(box1), key=lambda region: region.area)
    box1_bbox = box1_props.bbox

    # Apply another morphological opening operation with an octagon.
    box2 = binary_opening(binary, octagon(9, 1))
    box2 = label(box2)
    box2_props = max(regionprops(box2), key=lambda region: region.area)
    box2_bbox = box2_props.bbox

    # Calculate the coordinates of the region of interest (ROI).
    roi = (box2_bbox[1], box1_bbox[0], box2_bbox[3] - box2_bbox[1], box1_bbox[2] - box1_bbox[0])
    roi = tuple(map(int, roi))

    # Crop the region of interest (ROI) from the original image.
    fov = img[new_y0:, :, :]
    fov = fov[roi[1]:roi[1] + roi[3], roi[0]:roi[0] + roi[2], :]
    
    return fov

In [3]:
def usDcallipers(impath):
    
    if impath.endswith(".bmp"):
        factor=15
    elif impath.endswith(".jpg") or impath.endswith(".jpeg"):
        factor=45
        
    f=cv2.imread(impath)

    # Difference between red and blue channels and normalization
    imagen=usFOV(f)
    mg1 = np.abs(imagen[:,:,0] - imagen[:,:,2])
    mg1 = (mg1 - np.min(mg1)) / (np.max(mg1) - np.min(mg1))

    # Thresholding with a high value to identify only crosses and ignore other artifacts
    cm_init = mg1 > 0.55
    
    # Opening and closing to have a better approximation of a cross shape
    se1 = np.ones((1, 2))
    se2 = np.ones((9, 9))

    cm_init = remove_small_objects(cm_init, factor)

    # Perform the morphological opening operation.
    cm = binary_opening(cm_init, se1)

    # Perform the morphological closing operation.
    cm = binary_closing(cm,se2)
    h, w = cm.shape

    # Remove artifacts in defined regions to prevent erroneous calliper detection 
    cm[0:int(h*0.25), int(w*(1-0.15)):] = 0  # Region 1
    cm[int(h*(0.9)):, :] = 0                # Region 2
    cm[:, 0:int(w*0.10)] = 0   

    cm = remove_small_objects(cm, 3)
    cm= cm.astype(np.uint8)
    labeled_cm = label(cm)

    if np.max(labeled_cm) > 2:
        # Filter to keep only the three largest connected components.
        cm = cm.astype(bool)
        cm = remove_small_objects(cm, 3)
        labeled_cm = label(cm)

        # Obtain the properties of the remaining regions.
        regions = regionprops(labeled_cm)

        # Initialize a list to store the centroids of the remaining regions.
        centroids = []

        # Iterate over the regions and obtain the centroids.
        for region in regions:
            centroids.append(region.centroid)

        # Convert the list of centroids into a NumPy array.
        centroids = np.array(centroids)

        # Calculate the distances between the centroids.
        percs = pdist(centroids) / np.sqrt(w ** 2 + h ** 2)
        cond = np.any((percs[percs > 0.02]) < 0.07)

    # Obtaining calliper coordinates
    labeled_cm = label(cm)

    p1=[0,0]
    p2=[0,0]

    # Labeled regions.
    regions = regionprops(labeled_cm)
    prueba=len(np.unique(labeled_cm))
    
    # Find the coordinates of the labeled regions.
    for region in regions:
        if region.label == prueba-1:
            y1, x1 = region.coords[:, 0], region.coords[:, 1]
            p1 = [np.mean(x1), np.mean(y1)]
        elif region.label == prueba-2:
            y2, x2 = region.coords[:, 0], region.coords[:, 1]
            p2 = [np.mean(x2), np.mean(y2)]

   # p1 always above p2.
    if p2[1] - p1[1] < 0:
        p1, p2 = p2, p1

    return cm, p1, p2,imagen

In [4]:
def usPreprocess(f):
    
    cm, p1, p2,f1 = usDcallipers(f)
    gray = cv2.cvtColor(f1, cv2.COLOR_RGB2GRAY)
    _, bm = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    # Background
    bm = fillHoles(bm)
    bm = cv2.erode(bm, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
    bm = cv2.morphologyEx(bm, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    
    # Normalization, G and B channels.
    mg2 = np.abs(f1[:,:,0] - f1[:,:,1])
    mg2 = (mg2 - np.min(mg2)) / (np.max(mg2) - np.min(mg2))
    
    am_init = mg2 > 0.1
    dam_init = cv2.dilate(am_init.astype(np.uint8), np.ones((2, 2), np.uint8))
    
    if cm.shape != dam_init.shape:
        dam_init = cv2.resize(dam_init, (cm.shape[1], cm.shape[0]))
    
    # Perform the logical OR operation.
    am = np.logical_or(cm, dam_init)
    
    # Convert image to a single channel.
    rimg = np.mean(f1[:, :, 0:2], axis=2).astype(np.uint8)
    
    # Create a dilated region mask.
    dregion_mask = cv2.dilate(am_init.astype(np.uint8), np.ones((2, 2), np.uint8))
    
    # Inpaint the image.
    g = cv2.inpaint(rimg, dregion_mask, 3, cv2.INPAINT_TELEA)

    return g, am, bm, p1, p2

def fillHoles(bm):
    # Find the contours in the binary image.
    contours, _ = cv2.findContours(bm, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on a black image.
    filled = np.zeros_like(bm)
    cv2.drawContours(filled, contours, -1, 255, thickness=cv2.FILLED)

    return filled

In [35]:
def usSeg(cm, p1, p2):
    seg = np.zeros_like(cm)
    
    p1 = (int(p1[0]), int(p1[1]))
    p2 = (int(p2[0]), int(p2[1]))
    angle = 150

    angle_rad = math.radians(angle)
    a = 60

    p1_up = (int(p1[0] + a * math.cos(angle_rad)), int(p1[1] - a * math.sin(angle_rad))) 
    p1_down = (int(p1[0] - (a) * math.cos(angle_rad)), int(p1[1] + (a) * math.sin(angle_rad)))

    for y in range(p2[1], -1, -1):
        distance_from_p2_up=100
        if cm[y, p2[0]] == 0:
            distance_from_p2_up = p2[1] - y  
            break
    #if distance_from_p2_up != -1:
        p2_down = (p2[0], p2[1] + distance_from_p2_up)

    p1_vertical_up = (p1[0], 0)
    p2_vertical_up = (p2[0], 0)

    cv2.line(seg, p1, p1_up, color=1)  
    cv2.line(seg, p1, p1_down, color=1)  
    cv2.line(seg, p2, p2_down, color=1)  
    cv2.line(seg, p2, p2_vertical_up, color=1)

    curve_points = []
    P0 = np.array(p2_down)
    P1 = np.array(p1_down)
    
    control_point = ((P0 + P1)/2).astype(int) 
    control_point = (control_point[0]+100, control_point[1]+10)  
    
    for t in np.linspace(0, 1, 40):
        B = (1-t)**2 * P0 + 2*(1-t)*t*np.array(control_point) + (t**2)*P1
        curve_points.append(tuple(B.astype(int)))



    for i in range(1, len(curve_points)):
        cv2.line(seg, curve_points[i - 1], curve_points[i], color=1)

    polygon_points = np.array([p1_up, p2_vertical_up, p2, p2_down] + curve_points[::-1] + [p1_down, p1], dtype=np.int32)
    cv2.fillPoly(seg, [polygon_points], color=1)
    
    seg = seg * (cm == 255).astype(np.uint8)

    shift = 8
    seg = np.pad(seg, ((shift, 0), (0, 0)), mode='constant')[:-shift, :]


    return seg

In [19]:
# Ruta base de imágenes y Excel
ruta_base = r"C:\Users\Mafe Valenzuela\Documents\Cps\01"
ruta_excel = r"C:\Users\Mafe Valenzuela\Documents\CPS\01\data\dataset.xlsx"

# Carpetas de salida
output_folder_pre = os.path.join(ruta_base, "imagenes_preprocesadas")
output_folder_seg = os.path.join(ruta_base, "imagenes_segmentadas")
os.makedirs(output_folder_pre, exist_ok=True)
os.makedirs(output_folder_seg, exist_ok=True)

# Leer Excel y filtrar categoría 3
df = pd.read_excel(ruta_excel)

# Lista para guardar coordenadas
callipers_data = []

# Procesar cada imagen de categoría 3
for _, row in tqdm(df.iterrows(), total=len(df), desc="Procesando imágenes"):
    # Construir ruta completa
    image_path = os.path.join(ruta_base, row["medical_center"], row["path"])
    
    if not os.path.exists(image_path):
        print(f"No encontrada: {image_path}")
        continue
    
    try:
        g, am, bm, p1, p2 = usPreprocess(image_path)
        
        seg = usSeg(bm , p1, p2)  # cm multiplicado por 255 para que sea binario válido
        
        # Guardar imagen preprocesada
        filename = os.path.basename(image_path)
        cv2.imwrite(os.path.join(output_folder_pre, filename), g)
        
        # Guardar imagen segmentada
        seg_path = os.path.join(output_folder_seg, filename)
        cv2.imwrite(seg_path, seg * 255)  # escala binaria a 0-255
        
        # Guardar coordenadas
        callipers_data.append({
            "imagen": filename,
            "p1_x": p1[0], "p1_y": p1[1],
            "p2_x": p2[0], "p2_y": p2[1]
        })
    except Exception as e:
        print(f"Error procesando {image_path}: {e}")

# Guardar coordenadas en CSV
df_coords = pd.DataFrame(callipers_data)
df_coords.to_csv(os.path.join(ruta_base, "callipers_positions.csv"), index=False)

print("Procesamiento completado.")


Procesando imágenes:   0%|                                                                     | 0/297 [00:00<?, ?it/s]C:\Users\Mafe Valenzuela\AppData\Local\Temp\ipykernel_14652\1366226959.py:42: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  cm = remove_small_objects(cm, 3)
Procesando imágenes: 100%|███████████████████████████████████████████████████████████| 297/297 [31:51<00:00,  6.43s/it]


Procesamiento completado.
